In [1]:
import os

try:
    os.mkdir("./data")

except:
    pass


!wget https://raw.githubusercontent.com/dmlc/web-data/master/mxnet/tinyshakespeare/input.txt -P ./data

--2023-05-09 02:16:05--  https://raw.githubusercontent.com/dmlc/web-data/master/mxnet/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘./data/input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.03s   

2023-05-09 02:16:06 (32.7 MB/s) - ‘./data/input.txt’ saved [1115394/1115394]



In [2]:
import torch
import torch.nn as nn

In [3]:
!pip install unidecode

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 kB 2.8 MB/s eta 0:00:00


In [4]:
import unidecode
import string
import random
import re
import time, math

In [20]:
num_epochs = 2000
print_every = 100
plot_every = 10

chunk_len = 200

hidden_size = 200
batch_size = 1
num_layers = 1
embedding_size = 70
lr = 0.002

In [21]:
all_characters = string.printable #위에 import 해준거 all_characters로 받아주고,

n_characters = len(all_characters) 
print(all_characters)
print('num_chars = ', n_characters)

0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~ 	

num_chars =  100


In [22]:
file = unidecode.unidecode(open('./data/input.txt').read())
file_len = len(file)
print('file_len =', file_len) #총 1115394개의 문자

file_len = 1115394


In [23]:
def random_chunck():
    start_index = random.randint(0, file_len - chunk_len) # 전체 파일에서 200개 빼준거, start_index로
    end_index = start_index + chunk_len + 1 # start_index 받아준거에서 그 이후꺼.
    
    return file[start_index : end_index]

print(random_chunck())
 #값은 랜덤하게 불러와서 맨날 바뀜.

:
Very well; and could be content to give him good
report fort, but that he pays himself with being proud.

Second Citizen:
Nay, but speak not maliciously.

First Citizen:
I say unto you, what he hath 


In [24]:
#문자열을 인덱스로 반환하는 함수.

def char_tensor(string):
    tensor = torch.zeros(len(string)).long() #string길이만큼 0으로 채우고, 100개.
    
    for c in range(len(string)):
        tensor[c] = all_characters.index(string[c]) #반복문 돌면서 0 아닌 값으로 채워넣기.
    return tensor

print(char_tensor('ABCdef')) #이놈들이 몇번째인지 확인.

tensor([36, 37, 38, 13, 14, 15])


In [25]:
def random_training_set():
    chunck = random_chunck()
    inp = char_tensor(chunck[: -1]) #맨 뒷글자 빼고 전부.
    target = char_tensor(chunck[1:]) #2번째 글자부터.
    # ex) 문자 ryuch이면, 첨에 ryuc입력, 목표(타겟)값은 yuch. 이런 느낌.
    return inp, target

In [26]:
class RNN(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, output_size, num_layers = 1):
        super(RNN, self).__init__()
        self.input_size = input_size
        self.embedding_size = embedding_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.num_layers = num_layers

        self.encoder = nn.Embedding(self.input_size, self.embedding_size) # 임베딩 작업 수행.(어떤 확률값으로 바꾸고)
        
        self.rnn = nn.RNN(self.embedding_size, self.hidden_size, self.num_layers) 
        
        self.decoder = nn.Linear(self.hidden_size, self.output_size) # 임베딩 작업 후 나온 확률값 바탕으로 출력 시퀀스 생성.

    def forward(self, input, hidden):
        out = self.encoder(input.view(1, -1))
        out, hidden = self.rnn(out, hidden)
        out = self.decoder(out.view(batch_size, -1)) #view가 텐서 크기 변경하는 함수.

        return out, hidden

    def init_hidden(self):
        hidden = torch.zeros(self.num_layers, batch_size, self.hidden_size)
        return hidden

model = RNN(n_characters, embedding_size, hidden_size, n_characters, num_layers)

In [27]:
model = RNN(input_size = n_characters,
            embedding_size = embedding_size,
            hidden_size = hidden_size,
            output_size = n_characters,
            num_layers = 2)

In [28]:
inp = char_tensor('A') #아까 위에서 선언 해준것. (문자열 인덱스로 바꾸는 것.)
print(inp)

hidden = model.init_hidden() #은닉층 거쳐서 초기화
print(hidden.size())

out, hidden = model(inp, hidden) #rnn 돌고 은닉층 거치고.
print(out.size())

tensor([36])
torch.Size([2, 1, 200])
torch.Size([1, 100])


In [29]:
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
loss_func = nn.CrossEntropyLoss()

In [30]:
def test():
    start_str = "b"
    inp = char_tensor(start_str)
    hidden = model.init_hidden()
    x = inp

    print(start_str, end="")
    for i in range(200):
        output,hidden = model(x, hidden)

        output_dist = output.data.view(-1).div(0.8).exp()
        top_i = torch.multinomial(output_dist, 1)[0] # 매번 같은 값만 아니고, 랜덤하게 다음 글자 뽑으려고 'multinomial'
        predicted_char = all_characters[top_i]

        print(predicted_char, end = "")

        x = char_tensor(predicted_char)

In [31]:
for i in range(num_epochs):
    inp, label = random_training_set()
    hidden = model.init_hidden()

    loss = torch.tensor([0]).type(torch.FloatTensor)
    optimizer.zero_grad()

    for j in range(chunk_len -1):
        x = inp[j] #반복문 돌면서 x값으로 넣고,
        y_ = label[j].unsqueeze(0).type(torch.LongTensor) #라벨값 가져오고 LongTensor형태로 바꾼 것.
        y, hidden = model(x, hidden)
        loss += loss_func(y, y_)

    loss.backward()
    optimizer.step()

    if i % 100 == 0:
        print("\n", loss / chunk_len, '\m') #loss가 chunk_len(200)만큼 더해져서 이렇게 나눠줌.
        test() 
        print('\m', '=' * 100)


 tensor([4.5635], grad_fn=<DivBackward0>) \m
Q*kOc.T_vtdzrTB=b)k_*\/ImHSZ9.o?6F
$ysr>tk JgItDLE,)bb,.TyO7:v`KP/R)
js9*P{Nrh
'KQ&Um)N%ZK<?}tRfD@%Tz6&+
iWx 0gi26h	O*a{GrLM	5VRd\m ====================================================================================================

 tensor([2.4152], grad_fn=<DivBackward0>) \m
bphad, ihd bewe ylith borl'y Core w?ndl
Ancenrls, wouFhoull hane dis wy and the the seas touch Ms wo lis,
Sule Whase is wake sarthit ighes ath fand mace ciwher, as his it hirs cnpathe fside were byow, \m ====================================================================================================

 tensor([2.5757], grad_fn=<DivBackward0>) \m
bear
ouk indy of pobe for bove mind pthes soar proan tou dat; this, bese, my your afe woO
The my ind vord agith mise wyoues hat,
Thich for sead mothh so sone wim to I mof,
Nnd artede of mer mone I ast \m ====================================================================================================

 tensor([2.1